In [ ]:
import torch
from torch import nn
from torch.utils.data import DataLoader
import matplotlib.pyplot as plt
import math
from torchvision import datasets, utils, transforms

In [ ]:
class DeepKernel(nn.Module):
    def __init__(self, in_channel: int,kernel_size: int) -> None:
        super().__init__()
        self.kernel = nn.Sequential(
            nn.Flatten(),
            nn.Linear(in_features=kernel_size ** 2 * in_channel, out_features= 32),
            nn.ReLU(),
            nn.Linear(in_features=32, out_features=1)
        )
        self.kernel_size = kernel_size
        self.in_channel = in_channel
    def forward(self, x: torch.Tensor):
        return self.kernel(x)

In [ ]:
class DeepConv2d(nn.Module):
    def __init__(self, in_channels: int, out_channels: int, kernel_size: int,stride: int = 1, padding: int = 0) -> None:
        super().__init__()
        self.in_channels = in_channels
        self.out_channels = out_channels
        self.kernel_size = kernel_size
        self.stride = stride
        self.padding = padding
        self.deepkernel: list[DeepKernel] = []
        for i in range(out_channels):
            self.deepkernel.append(DeepKernel(in_channel=self.in_channels,kernel_size=kernel_size))
    def forward(self, x: torch.Tensor):
        batch_size, _, height, width = x.shape
        new_height = math.floor((height + 2 * self.padding - (self.kernel_size - 1) - 1) / self.stride) + 1
        new_widht = math.floor((width + 2 * self.padding - (self.kernel_size - 1) - 1) / self.stride) + 1
        ret = torch.empty(batch_size, self.out_channels, new_height, new_widht )
        for out_channel in range(self.out_channels):
            # deep convolution loop
            for h in range((height - (self.kernel_size - 1))):
                for w in range((height - (self.kernel_size - 1))):
                    xx = x[:, :, h:self.kernel_size+h, w:self.kernel_size+w]
                    ret[:, :, 1, 1] = self.deepkernel[out_channel](x[:, :, h:h+self.kernel_size, w:w+self.kernel_size])
        return ret

In [ ]:
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])
train_dataset = datasets.CIFAR100("./data", train=True, download=True, transform=transform)

In [ ]:
class DeepCNN(nn.Module):
    def __init__(self) -> None:
        super().__init__()
        # (batch_size, 3, 32, 32)
        self.dconv1 = DeepConv2d(in_channels=3, out_channels=16, kernel_size=3) # (16, 30, 30)
        self.dconv2 = DeepConv2d(in_channels=16, out_channels=32, kernel_size=3) # (32, 28, 28)
        self.dconv3 = DeepConv2d(in_channels=32, out_channels=64, kernel_size=3) # (32, 26, 26)
        self.fc1 = nn.Linear(in_features=64 * 26 * 26, out_features=120)
        self.fc2 = nn.Linear(120, 100)
        self.relu = nn.ReLU()
    def forward(self, x):
        x = self.dconv1(x)
        x = self.relu(x)
        x = self.dconv2(x)
        x = self.relu(x)
        x = self.dconv3(x)
        x = self.relu(x)
        x = x.view(x.shape[0], -1)
        x = self.fc1(x)
        x = self.relu(x)
        x = self.fc2(x)
        return x
class CNN(nn.Module):
    def __init__(self) -> None:
        super().__init__()
        # (batch_size, 3, 32, 32)
        self.dconv1 = nn.Conv2d(in_channels=3, out_channels=16, kernel_size=3) # (16, 30, 30)
        self.dconv2 = nn.Conv2d(in_channels=16, out_channels=32, kernel_size=3) # (32, 28, 28)
        self.dconv3 = nn.Conv2d(in_channels=32, out_channels=64, kernel_size=3) # (32, 26, 26)
        self.fc1 = nn.Linear(in_features=64 * 26 * 26, out_features=120)
        self.fc2 = nn.Linear(120, 100)
        self.relu = nn.ReLU()
    def forward(self, x):
        x = self.dconv1(x)
        x = self.relu(x)
        x = self.dconv2(x)
        x = self.relu(x)
        x = self.dconv3(x)
        x = self.relu(x)
        x = x.view(x.shape[0], -1)
        x = self.fc1(x)
        x = self.relu(x)
        x = self.fc2(x)
        return x